# Extractor

>
> A Libraray to extract logits from the last layer of a hugginface transformer.
>

In [ ]:
#| default_exp extractor

In [2]:
#| export
import pandas as pd
import os
import time
import numpy as np
import torch
import gc

In [2]:
#| export
import logging
logging.basicConfig(
    filename="logger_extractor.txt",
    filemode='a',
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO
    )

In [4]:
#| export
torch.__version__

'1.12.1+cu113'

In [5]:
#| export
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
#| hide
#device = 'cpu' #Forcing CPU

In [7]:
#| hide
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.0375, 0.7329, 0.9940],
        [0.6435, 0.5687, 0.1543],
        [0.5168, 0.4924, 0.0014],
        [0.1344, 0.8388, 0.7580],
        [0.8156, 0.1902, 0.2370]])


In [8]:
#| hide
x.get_device()

-1

In [9]:
#| hide
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [10]:
#| hide
torch.cuda.memory_allocated()

0

In [11]:
#| hide
#! nvidia-smi

Sun Apr 23 01:18:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   32C    P0    33W / 250W |      7MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model
>

In [12]:
#| export
def c_eleuther( model_type, returnModel = False, cache_path = "../datax/cache"):
    ''' Eleuther and Salesforce and Parrot uses the same importation'''
    from transformers import AutoTokenizer, AutoModelForCausalLM
    tokenizer = AutoTokenizer.from_pretrained(model_type, cache_dir = cache_path )
    model = []
    logging.info("Tokenizer Loaded")
    if returnModel:
        model = AutoModelForCausalLM.from_pretrained(model_type, cache_dir = cache_path)
        logging.info("Model Loaded")        
    
    logging.info(model_type)
    return tokenizer, model
    
    
    

def init_model_args( current_case = 'c1', returnModel = False ): 
    
    code_models = {
        'c1':('EleutherAI/gpt-neo-125m', ), # Basic (on Pile) GPT-3/J
        'c2':('EleutherAI/gpt-neo-1.3B', ),
        'c3':('EleutherAI/gpt-neo-2.7B', ),
        'c4':('EleutherAI/gpt-j-6b', ),
        'c5':('Salesforce/codegen-350M-nl', ), #Basic (on Pile) codegen
        'c6':('Salesforce/codegen-2B-nl', ),
        'c7':('Salesforce/codegen-6B-nl', ),
        'c8':('Salesforce/codegen-16B-nl', ),
        'c9':('codeparrot/codeparrot-small-multi', ), #multi-Language
        'c10':('Salesforce/codegen-350M-multi', ),
        'c11':('Salesforce/codegen-2B-multi', ),
        'c12':('Salesforce/codegen-6B-multi', ),
        'c13':('Salesforce/codegen-16B-multi', ),
        'c14':('codeparrot/codeparrot-small', ), #mono-Language
        'c15':('codeparrot/codeparrot', ),
        'c16':('Salesforce/codegen-350M-mono', ),
        'c17':('Salesforce/codegen-2B-mono', ),
        'c18':('Salesforce/codegen-6B-mono', ),
        'c19':('Salesforce/codegen-16B-mono', ),
    }
    
    model_type = code_models[current_case][0]
    tokenizer, model =  c_eleuther( returnModel = returnModel,  model_type = model_type ) 
    
    
    return model_type, tokenizer, model

# Init Parameters
> Loading Models and Testbeds


In [13]:
#| hide
# [WARNING] Hyperparameters changes, please tune them up
params = {
    'codemodel' : 'c4',
    'numpy_files_logits_path': '../datax/np_files_logits/c4',
    'testbeds_path' : '../datax/testbeds/AstEvalVerticalFiltered.json'
}

In [14]:
#| hide
#Testing data loads
data_pd = pd.read_json( params['testbeds_path'] )
data_pd.head(1)

,size,ids,m_name,code,ast_errors,n_ast_errors,ast_levels,n_whitespaces_,complexity,nloc,token_counts,n_ast_nodes
0,280,"[4299, 1057, 62, 29412, 62, 41989, 7, 9288, 62...",c1,"def run_python_tests(test_modules, parallelism...",[],0,13,67,7,15,120,154


In [15]:
#| hide
#Uploading Model UnderAnalisys
name, tokenizer, model = init_model_args(
    current_case = params['codemodel'], 
    returnModel = True #[WARNING!] Check the parameters before calling it. 
    )

# Extracting Logits From a Given Model

### Creating data tensors

In [16]:
#| hide
filtered_prompts_ids = data_pd[data_pd['m_name']==params['codemodel']]['ids'].values

In [17]:
#| hide
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

tf_input_ids = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in filtered_prompts_ids ]

### Loading Model to Memory

In [18]:
#| hide
model.to( device ) #WARNING, Verify the device before assigning to memory

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
  

### Executing Logits

In [19]:
#| export
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [20]:
#| export
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast
    CODEMODEL =  params['codemodel']
    create_folder(params['numpy_files_logits_path'])

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        output = [ model( 
            input_ids = i, 
            labels = i.type(torch.LongTensor).to(device) 
            ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
    
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( params['numpy_files_logits_path']+ '/'+ f'logits_tensor[{jdx+n}]_batch[{current_batch}]_model[{CODEMODEL}].npy', o_logits) #Saving LOGITS
        np.save( params['numpy_files_logits_path']+ '/'+f'loss_batch[{current_batch}]_model[{CODEMODEL}].npy', output_loss) #Saving LOSS
        
        logging.info(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:
            del out.logits
            torch.cuda.empty_cache()
            del out.loss
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
        del output
        del output_logits
        del output_loss
    
    pass

In [21]:
#| hide
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(
    batch = 1, 
    input = tf_input_ids, 
    from_index=0
)

KeyboardInterrupt: 